In [ ]:
#GP prior 먼저 활용(squared exponential covariance kernal)
#log normal likelihood
pyro.clear_param_store()
kernel = gp.kernels.RBF( #RBF kernel
    ard_num_dims=1,  # Automatic Relevance Determination for 1D input
    input_dim = 1, variance =torch.tensor(5.0),
    lengthscale = torch.tensor(10.0)
)#초깃값

gpr = gp.models.GPRegression(X, y ,kernel , noise=torch.tensor(1.0))#가우시안 확률과정 이미 개발된 모형

#noge that our priors have support on the positive reals
gpr.kernel.lengthscale = pyro.nn.PyroSample(dist.Lognormal(0.0, 1.0))
gpr.kernel.variance = pyro.nn.PyroSample(dist.Lognormal(0.0, 1.0))
gpr.noise = pyro.nn.PyroSample(dist.Lognormal(0.0, 1.0))
#GP prior를 활용한 모델링


In [ ]:
#변분 방법을 이용한 모델 학습
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
#변분 추론을 위한 최적화 알고리즘   

optim = Adam({"lr": 0.01}) #학습률 설정
svi = SVI(gpr.model, gpr.guide, optim, loss=Trace_ELBO()) #변분 추론 객체 생성
losses = [] #손실값을 저장할 리스트
variance = [] #변동성을 저장할 리스트
lengthscales = [] #길이 척도를 저장할 리스트
noises = [] #노이즈를 저장할 리스트
#학습을 위한 반복문
num_steps = 1000 if not smoke_test else 2
for i in range(num_steps):
    variances.append(gpr.kernel.variance.item())
    noises.append(gpr.noise.item())
    lengthscales.append(gpr.kernel.lengthscale.item())
    losses.append(svi.step())  # X와 y를 이용하여 SVI를 수행

In [ ]:
#SGP prior 이용한 모델 , conditional 분포를 이용한 예측
#initalize the inducing inputs

Xu = torch.arrange(20.0)/ 4.0

pyro.clear_param_store()
kernel = gp.kernesl.RBF(input_dim =1 , variance = torch.tensor(5.0),
                        lengthscale = torch.tensor(10.0))
sgpr - gp.models.SparseGPRegression(X, y ,kernel ,Xu = Xu , jitter =1.0e-5) #Sparse GP Regression 모델
#잡음 필요해서 jitter 추가
#note that our priors have support on the positive reals
sgpr.kernel.lengthscale = pyro.nn.PyroSample(dist.Lognormal(0.0, 1.0))
sgpr.kernel.variance = pyro.nn.PyroSample(dist.Lognormal(0.0, 1.0))
sgpr.noise = pyro.nn.PyroSample(dist.Lognormal(0.0, 1.0))



In [ ]:
#GP regression -fitting
svi = SVI(sgpr.model, sgpr.guide, optim, loss=Trace_ELBO()) #모듈에서 주어진 guide를 이용할 수도 있고, 바꾸고 싶으면 guide 직접 정의도 가능
losses = []  # 손실값을 저장할 리스트
variances = []  # 변동성을 저장할 리스트
locations = []  # 위치를 저장할 리스트
lengthscales = []  # 길이 척도를 저장할 리스트
noises = []  # 노이즈를 저장할 리스트
# 학습을 위한 반복문
num_steps = 1000 if not smoke_test else 2
for i in range(num_steps):
    locations.append(sgpr.Xu.data.numoy().copy()) #locations는 Xu의 위치를 저장 , 어떤 input이 최적인지 찾기 위한 구문문
    variances.append(sgpr.kernel.variance.item())
    noises.append(sgpr.noise.item())
    lengthscales.append(sgpr.kernel.lengthscale.item())
    losses.append(svi.step())  # X와 y를 이용하여 SVI를 수행
    